<a href="https://colab.research.google.com/github/castudil/bacteria-multi-label/blob/main/multilabel_bac.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Libraries used

In [1]:
import os
import copy

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import (f1_score, multilabel_confusion_matrix,
                             accuracy_score, hamming_loss, jaccard_score, make_scorer)
from sklearn.model_selection import train_test_split, RepeatedKFold

import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

import keras_tuner as kt
import keras

from joblib import dump, load
import joblib

2023-12-18 17:18:29.594101: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-12-18 17:18:29.631395: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-18 17:18:29.631422: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-18 17:18:29.632397: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-18 17:18:29.639118: I tensorflow/core/platform/cpu_feature_guar

In [2]:
tf.random.set_seed(0)
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Data Loading

In [3]:
os.chdir("..")

In [4]:
train_file = "data/processed/raw/train_s_aureus_driams.csv"
train_bac = pd.read_csv(train_file)
train_bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9993,9994,9995,9996,9997,9998,9999,Oxacillin,Clindamycin,Fusidic acid
0,0.018721,0.016147,0.016983,0.021218,0.020846,0.019784,0.019405,0.023356,0.026224,0.026569,...,0.037966,0.030364,0.037545,0.040851,0.034176,0.046110,0.025638,0.0,0.0,0.0
1,0.009001,0.007475,0.006874,0.008575,0.009539,0.007894,0.008314,0.008013,0.008664,0.008923,...,0.014496,0.024966,0.027437,0.026541,0.022940,0.020572,0.032504,0.0,0.0,0.0
2,0.022354,0.020220,0.020910,0.024631,0.021436,0.021197,0.020229,0.018818,0.018637,0.018815,...,0.024620,0.022942,0.026715,0.032045,0.030431,0.029085,0.013117,0.0,0.0,0.0
3,0.017619,0.016073,0.016407,0.018011,0.019364,0.018950,0.017607,0.019116,0.023623,0.024492,...,0.051312,0.047458,0.049338,0.055039,0.054541,0.058643,0.058919,0.0,0.0,0.0
4,0.008264,0.008229,0.006753,0.006657,0.010107,0.007039,0.008250,0.010670,0.008134,0.006513,...,0.236769,0.217499,0.187244,0.216243,0.221910,0.226531,0.221965,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2819,0.056616,0.039011,0.040380,0.048517,0.050865,0.047771,0.049312,0.048257,0.049417,0.049000,...,0.021169,0.023617,0.033694,0.021037,0.018727,0.010641,0.009238,0.0,1.0,0.0
2820,0.125837,0.107712,0.109186,0.107613,0.109855,0.105060,0.099640,0.104169,0.120303,0.125067,...,0.082375,0.083446,0.096510,0.084883,0.092228,0.085599,0.042142,0.0,1.0,0.0
2821,0.000000,0.000000,0.035603,0.039994,0.042372,0.046666,0.045781,0.043914,0.039875,0.037170,...,0.006903,0.008322,0.011071,0.010274,0.004682,0.003547,0.001744,0.0,0.0,0.0
2822,0.005443,0.005998,0.003670,0.005588,0.006124,0.005019,0.004853,0.005400,0.004169,0.005151,...,0.049241,0.039586,0.050542,0.039139,0.046816,0.043036,0.037402,1.0,1.0,0.0


In [5]:
test_file = "data/processed/raw/test_s_aureus_driams.csv"
test_bac = pd.read_csv(test_file)
test_bac

,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,...,9993,9994,9995,9996,9997,9998,9999,Oxacillin,Clindamycin,Fusidic acid
0,0.044453,0.032486,0.032540,0.034223,0.037528,0.039503,0.031378,0.035506,0.037688,0.035658,...,0.247814,0.263833,0.279904,0.264432,0.241573,0.266020,0.231517,0.0,0.0,0.0
1,0.004318,0.001881,0.001274,0.000902,0.000892,0.000049,0.002188,0.002001,0.003081,0.003384,...,0.033364,0.042735,0.066426,0.057485,0.052903,0.050839,0.036631,0.0,0.0,0.0
2,0.026184,0.026459,0.025393,0.028609,0.031314,0.031739,0.033337,0.028051,0.028047,0.028978,...,0.086746,0.087719,0.094103,0.080969,0.073970,0.069047,0.070988,0.0,1.0,0.0
3,0.000000,0.015010,0.017782,0.014582,0.015084,0.018046,0.014461,0.014400,0.018769,0.018272,...,0.005062,0.006748,0.004573,0.007583,0.008427,0.004729,0.002394,0.0,0.0,1.0
4,0.060499,0.043034,0.030296,0.032635,0.031272,0.032762,0.031154,0.030382,0.030233,0.029438,...,0.029452,0.033288,0.039230,0.046477,0.037219,0.022227,0.019920,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
702,0.013092,0.011730,0.009902,0.013513,0.015317,0.011370,0.013338,0.010117,0.010994,0.009222,...,0.050851,0.046334,0.042840,0.061644,0.066948,0.061007,0.056178,0.0,0.0,0.0
703,0.021222,0.015896,0.015512,0.017995,0.018663,0.019427,0.018667,0.014589,0.016765,0.015071,...,0.047285,0.056455,0.058002,0.055528,0.043539,0.039962,0.034761,1.0,0.0,0.0
704,0.045613,0.041040,0.046177,0.050216,0.046525,0.048843,0.045920,0.044283,0.043983,0.046330,...,0.104234,0.094692,0.090012,0.096624,0.092228,0.100024,0.043682,0.0,0.0,1.0
705,0.015193,0.011922,0.010877,0.009975,0.010474,0.012171,0.010417,0.010783,0.013170,0.014157,...,0.051772,0.058255,0.074609,0.068249,0.049157,0.070229,0.043615,1.0,0.0,0.0


In [6]:
train_x = train_bac[train_bac.columns.drop(list(train_bac.filter(regex='[^0-9]')))]
test_x = test_bac[test_bac.columns.drop(list(test_bac.filter(regex='[^0-9]')))]

In [7]:
antibiotics = train_bac.columns.drop(train_x.columns)

In [8]:
train_y = train_bac[antibiotics]
test_y = test_bac[antibiotics]

# Evaluation

In [9]:
def multilabel_f1_wrapper(true, pred, average="weighted"):
    if isinstance(true, list):
        true = np.array(true)
    elif isinstance(true, pd.DataFrame):
        true = true.to_numpy()
    if isinstance(pred, list):
        pred = np.array(pred)
    elif isinstance(true, pd.DataFrame):
        pred = pred.to_numpy()
    column = 0
    total = 0
    while column < true[0].size:
        total+=f1_score(true[:, column], pred[:, column], average=average)
        column+=1
    return total/(column)

In [10]:
def report(true, pred):
        
    hl = hamming_loss(true, pred)
    f1w = multilabel_f1_wrapper(true, pred, "weighted")
    acc = accuracy_score(true, pred)
    
    f1u = multilabel_f1_wrapper(true, pred, "macro")
    f1su = f1_score(true, pred, average="macro")
    f1sw = f1_score(true, pred, average="weighted")

    
    print("Main metrics:")
    print(" Hamming Loss:", hl)
    print(" Accuracy:", acc)
    print(" F1 Score (Weighted):", f1w)
    print("================================================")
    print("Other metrics:")
    print(" F1 Score (Unweighted):", f1u)
    print(" F1 Score (sklearn Unweighted):", f1su)
    print(" F1 Score (sklearn Weighted):", f1sw)
    return hl, acc, f1w

# Model

In [11]:
class BaseBacModel(Model):
  def __init__(self, input_dim, output_dim, units_1, units_2=0, units_3=0):
    super().__init__()
    self.d1 = Dense(units_1, input_dim=input_dim, kernel_initializer="he_uniform", activation="relu")
    
    self.use_d2 = units_2 > 0
    self.d2 = Dense(units_2, kernel_initializer="he_uniform", activation="relu")
    
    self.use_d3 = units_3 > 0
    self.d3 = Dense(units_3, kernel_initializer="he_uniform", activation="relu")
    
    self.d4 = Dense(output_dim, activation='sigmoid')

  def call(self, x):
    x = self.d1(x)
    if self.use_d2:
        x = self.d2(x)
    if self.use_d3:
        x = self.d3(x)
    return self.d4(x)

In [12]:
def evaluate_model(x, y):
    results = list()
    
    cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)
    
    np_x = x.to_numpy()
    np_y = y.to_numpy()
    
    for train_i, validation_i in cv.split(np_x):
        input_dim  = len(x.columns)
        output_dim = len(y.columns)
        model = BaseBacModel(input_dim, output_dim, 512)
        model.compile(loss="binary_crossentropy", optimizer="adam")
        
        train_x = np_x[train_i]
        train_y = np_y[train_i]
        validation_x = np_x[validation_i]
        validation_y = np_y[validation_i]

        model.fit(train_x, train_y, verbose=1, epochs=100)

        pred = model.predict(validation_x)
        pred = pred.round()
        
        hl, acc, f1w = report(validation_y, pred)

        results.append(f1w)
    return results

In [13]:
# evaluate_model(train_x, train_y)

2023-12-18 17:18:37.157284: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-12-18 17:18:37.157351: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: dslab
2023-12-18 17:18:37.157365: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: dslab
2023-12-18 17:18:37.157563: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 470.199.2
2023-12-18 17:18:37.157611: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 470.199.2
2023-12-18 17:18:37.157622: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 470.199.2


Epoch 1/100
71/71 [==============================] - 1s 7ms/step - loss: 0.4900
Epoch 2/100
71/71 [==============================] - 0s 6ms/step - loss: 0.4286
Epoch 3/100
71/71 [==============================] - 0s 6ms/step - loss: 0.4016
Epoch 4/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3949
Epoch 5/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3936
Epoch 6/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3651
Epoch 7/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3563
Epoch 8/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3438
Epoch 9/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3401
Epoch 10/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3394
Epoch 11/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3292
Epoch 12/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3349
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 1s 6ms/step - loss: 0.4706
Epoch 2/100
71/71 [==============================] - 0s 6ms/step - loss: 0.4217
Epoch 3/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3970
Epoch 4/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3816
Epoch 5/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3565
Epoch 6/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3465
Epoch 7/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3482
Epoch 8/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3454
Epoch 9/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3334
Epoch 10/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3305
Epoch 11/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3241
Epoch 12/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3226
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 1s 6ms/step - loss: 0.4663
Epoch 2/100
71/71 [==============================] - 0s 6ms/step - loss: 0.4142
Epoch 3/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3914
Epoch 4/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3794
Epoch 5/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3692
Epoch 6/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3552
Epoch 7/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3673
Epoch 8/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3500
Epoch 9/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3372
Epoch 10/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3431
Epoch 11/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3285
Epoch 12/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3299
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 1s 7ms/step - loss: 0.4775
Epoch 2/100
71/71 [==============================] - 0s 6ms/step - loss: 0.4355
Epoch 3/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3980
Epoch 4/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3964
Epoch 5/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3981
Epoch 6/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3799
Epoch 7/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3664
Epoch 8/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3568
Epoch 9/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3559
Epoch 10/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3477
Epoch 11/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3489
Epoch 12/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3350
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 1s 7ms/step - loss: 0.4708
Epoch 2/100
71/71 [==============================] - 0s 6ms/step - loss: 0.4356
Epoch 3/100
71/71 [==============================] - 0s 6ms/step - loss: 0.4046
Epoch 4/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3862
Epoch 5/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3791
Epoch 6/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3633
Epoch 7/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3596
Epoch 8/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3612
Epoch 9/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3415
Epoch 10/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3306
Epoch 11/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3442
Epoch 12/100
71/71 [==============================] - 0s 6ms/step - loss: 0.3400
Epoch 13/100
71/71 [=================

[0.883664695466931,
 0.8818226153563206,
 0.8632916291860679,
 0.8991638341199485,
 0.8746212758626853]

In [14]:
def cross_train(x, y, model):
    results = list()
    
    cv = RepeatedKFold(n_splits=5, n_repeats=1, random_state=1)
    
    np_x = x.to_numpy()
    np_y = y.to_numpy()
    
    for train_i, validation_i in cv.split(np_x):
        train_x = np_x[train_i]
        train_y = np_y[train_i]
        validation_x = np_x[validation_i]
        validation_y = np_y[validation_i]

        model.fit(train_x, train_y, verbose=1, epochs=100)

        pred = model.predict(validation_x)
        pred = pred.round()
        
        hl, acc, f1w = report(validation_y, pred)

        results.append(f1w)
    return results

In [15]:
# input_dim  = len(train_x.columns)
# output_dim = len(train_y.columns)
# model = BaseBacModel(input_dim, output_dim, 512)
# model.compile(loss="binary_crossentropy", optimizer="adam")
# cross_train(train_x, train_y, model)

In [16]:
# pred = model.predict(test_x)
# pred_binary = pred.round()

# hl, acc, f1w = report(test_y, pred_binary)

# Optimization

In [17]:
def model_builder(hp):
    
    hp_units_1 = hp.Int("units_1", min_value=32, max_value=640, step=32)
    hp_units_2 = hp.Int("units_2", min_value=0, max_value=640, step=32)
    hp_units_3 = hp.Int("units_3", min_value=0, max_value=640, step=32)

    hp_learning_rate = hp.Choice("learning_rate", values=[1e-2, 1e-3, 1e-4])
    hp_optimizer = hp.Choice("optimizer", ordered=False, values=[
        "Adam",
        "SGD"
    ])
    
    optimizer = keras.optimizers.Adam(learning_rate=hp_learning_rate)
    if hp_optimizer == "SGD":
        optimizer = keras.optimizers.SGD(learning_rate=hp_learning_rate)
    
    input_dim  = len(train_x.columns)
    output_dim = len(train_y.columns)
    
    model = BaseBacModel(input_dim, output_dim, hp_units_1, hp_units_2, hp_units_3)
    
    model.compile(
        optimizer=optimizer,
        loss="binary_crossentropy",
    )

    return model

In [18]:
tuner = kt.BayesianOptimization(
    model_builder,
    objective="val_loss",
    max_trials=1000,
)

Reloading Tuner from ./untitled_project/tuner0.json


In [19]:
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [20]:
tuner.search(train_x, train_y, epochs=150, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(best_hps.get('units_1'))
print(best_hps.get('units_2'))
print(best_hps.get('units_3'))
print(best_hps.get('optimizer'))
print(best_hps.get('learning_rate'))


Trial 1000 Complete [00h 00m 19s]
val_loss: 0.30658015608787537

Best val_loss So Far: 0.2977862060070038
Total elapsed time: 07h 42m 10s
640
352
640
Adam
0.001


In [21]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(train_x, train_y, epochs=150, validation_split=0.2)

val_acc_per_epoch = history.history["val_loss"]
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))


Epoch 1/150
71/71 [==============================] - 1s 12ms/step - loss: 0.4678 - val_loss: 0.4383
Epoch 2/150
71/71 [==============================] - 1s 10ms/step - loss: 0.4057 - val_loss: 0.4136
Epoch 3/150
71/71 [==============================] - 1s 11ms/step - loss: 0.3944 - val_loss: 0.3790
Epoch 4/150
71/71 [==============================] - 1s 11ms/step - loss: 0.3783 - val_loss: 0.3866
Epoch 5/150
71/71 [==============================] - 1s 12ms/step - loss: 0.3598 - val_loss: 0.3700
Epoch 6/150
71/71 [==============================] - 1s 11ms/step - loss: 0.3568 - val_loss: 0.4135
Epoch 7/150
71/71 [==============================] - 1s 9ms/step - loss: 0.3546 - val_loss: 0.3513
Epoch 8/150
71/71 [==============================] - 1s 10ms/step - loss: 0.3373 - val_loss: 0.4069
Epoch 9/150
71/71 [==============================] - 1s 10ms/step - loss: 0.3574 - val_loss: 0.3361
Epoch 10/150
71/71 [==============================] - 1s 9ms/step - loss: 0.3191 - val_loss: 0.3424
E

71/71 [==============================] - 1s 9ms/step - loss: 0.1099 - val_loss: 0.4876
Epoch 83/150
71/71 [==============================] - 1s 9ms/step - loss: 0.1234 - val_loss: 0.4637
Epoch 84/150
71/71 [==============================] - 1s 10ms/step - loss: 0.1046 - val_loss: 0.5167
Epoch 85/150
71/71 [==============================] - 1s 10ms/step - loss: 0.1095 - val_loss: 0.4916
Epoch 86/150
71/71 [==============================] - 1s 10ms/step - loss: 0.0915 - val_loss: 0.5570
Epoch 87/150
71/71 [==============================] - 1s 10ms/step - loss: 0.1037 - val_loss: 0.4883
Epoch 88/150
71/71 [==============================] - 1s 9ms/step - loss: 0.1157 - val_loss: 0.5210
Epoch 89/150
71/71 [==============================] - 1s 10ms/step - loss: 0.1033 - val_loss: 0.5887
Epoch 90/150
71/71 [==============================] - 1s 10ms/step - loss: 0.1093 - val_loss: 0.6129
Epoch 91/150
71/71 [==============================] - 1s 10ms/step - loss: 0.1390 - val_loss: 0.4979
Epoch 

In [22]:
model = tuner.hypermodel.build(best_hps)

# Retrain the model
cross_train(train_x, train_y, model)

Epoch 1/100
71/71 [==============================] - 1s 9ms/step - loss: 0.4680
Epoch 2/100
71/71 [==============================] - 1s 9ms/step - loss: 0.4140
Epoch 3/100
71/71 [==============================] - 1s 9ms/step - loss: 0.3864
Epoch 4/100
71/71 [==============================] - 1s 10ms/step - loss: 0.3698
Epoch 5/100
71/71 [==============================] - 1s 10ms/step - loss: 0.3660
Epoch 6/100
71/71 [==============================] - 1s 10ms/step - loss: 0.3573
Epoch 7/100
71/71 [==============================] - 1s 10ms/step - loss: 0.3382
Epoch 8/100
71/71 [==============================] - 1s 10ms/step - loss: 0.3249
Epoch 9/100
71/71 [==============================] - 1s 9ms/step - loss: 0.3261
Epoch 10/100
71/71 [==============================] - 1s 9ms/step - loss: 0.3241
Epoch 11/100
71/71 [==============================] - 1s 9ms/step - loss: 0.3169
Epoch 12/100
71/71 [==============================] - 1s 9ms/step - loss: 0.3035
Epoch 13/100
71/71 [============

Epoch 1/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1981
Epoch 2/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1388
Epoch 3/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1307
Epoch 4/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1252
Epoch 5/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1181
Epoch 6/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1389
Epoch 7/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1363
Epoch 8/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1249
Epoch 9/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1068
Epoch 10/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0959
Epoch 11/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1062
Epoch 12/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0830
Epoch 13/100
71/71 [=================

Epoch 1/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1338
Epoch 2/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0974
Epoch 3/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0858
Epoch 4/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0552
Epoch 5/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0721
Epoch 6/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0376
Epoch 7/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0570
Epoch 8/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0498
Epoch 9/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0471
Epoch 10/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0650
Epoch 11/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0412
Epoch 12/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0627
Epoch 13/100
71/71 [=================

18/18 [==============================] - 0s 2ms/step
Main metrics:
 Hamming Loss: 0.02182890855457227
 Accuracy: 0.9415929203539823
 F1 Score (Weighted): 0.9777231248250567
Other metrics:
 F1 Score (Unweighted): 0.9443077774143136
 F1 Score (sklearn Unweighted): 0.9013422051459458
 F1 Score (sklearn Weighted): 0.9257680397428959
Epoch 1/100
71/71 [==============================] - 1s 9ms/step - loss: 0.2120
Epoch 2/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0961
Epoch 3/100
71/71 [==============================] - 1s 8ms/step - loss: 0.1295
Epoch 4/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0712
Epoch 5/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0636
Epoch 6/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1278
Epoch 7/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0781
Epoch 8/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0299
Epoch 9/100
71/71 [==========

71/71 [==============================] - 1s 8ms/step - loss: 0.0988
Epoch 96/100
71/71 [==============================] - 1s 8ms/step - loss: 0.1114
Epoch 97/100
71/71 [==============================] - 1s 8ms/step - loss: 0.1076
Epoch 98/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0915
Epoch 99/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0749
Epoch 100/100
18/18 [==============================] - 0s 2ms/step
Main metrics:
 Hamming Loss: 0.06371681415929203
 Accuracy: 0.8371681415929203
 F1 Score (Weighted): 0.9333144957006367
Other metrics:
 F1 Score (Unweighted): 0.7818393564033919
 F1 Score (sklearn Unweighted): 0.5989227509455634
 F1 Score (sklearn Weighted): 0.6863279341283934
Epoch 1/100
71/71 [==============================] - 1s 9ms/step - loss: 0.1074
Epoch 2/100
71/71 [==============================] - 1s 8ms/step - loss: 0.1239
Epoch 3/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0924
Epoch 4/100
71/71 [====

71/71 [==============================] - 1s 8ms/step - loss: 0.0067
Epoch 92/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0037
Epoch 93/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0032
Epoch 94/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0030
Epoch 95/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0024
Epoch 96/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0015
Epoch 97/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0020
Epoch 98/100
71/71 [==============================] - 1s 8ms/step - loss: 0.0099
Epoch 99/100
71/71 [==============================] - 1s 9ms/step - loss: 0.0639
Epoch 100/100
18/18 [==============================] - 0s 2ms/step
Main metrics:
 Hamming Loss: 0.026004728132387706
 Accuracy: 0.9237588652482269
 F1 Score (Weighted): 0.9737328743334198
Other metrics:
 F1 Score (Unweighted): 0.9363477377275272
 F1 Score (sklearn Unweighted): 0.8880

[0.8914002600045383,
 0.9386203826897059,
 0.9777231248250567,
 0.9333144957006367,
 0.9737328743334198]

In [23]:
pred = model.predict(test_x)
pred_binary = pred.round()

hl, acc, f1w = report(test_y, pred_binary)

23/23 [==============================] - 0s 3ms/step
Main metrics:
 Hamming Loss: 0.11126826968411127
 Accuracy: 0.7114568599717115
 F1 Score (Weighted): 0.8842954732436407
Other metrics:
 F1 Score (Unweighted): 0.7145004880129727
 F1 Score (sklearn Unweighted): 0.49383523530602297
 F1 Score (sklearn Weighted): 0.5557422095354932
